In [ ]:
# default_exp core

# SGNN

> Implementation of Self-Governing Neural Networks for speech act classification

Implementation of the [SGNN paper](https://www.aclweb.org/anthology/D19-1402.pdf) for speech act classification.
This repository is inspired by Guillaume Chevalier's [implementation](https://github.com/guillaume-chevalier/SGNN-Self-Governing-Neural-Networks-Projection-Layer), as well as his [discussion](https://github.com/guillaume-chevalier/SGNN-Self-Governing-Neural-Networks-Projection-Layer/issues/1) with [Sava Kalbachou](https://github.com/thinline).
This version implements some things differently from Guillaume's code, and extends beyond the projection layer all the way to a fully trainable network.

The network is trained to classify the [SwDA corpus](https://web.stanford.edu/~jurafsky/ws97/) utterances according to their speech act. The corpus was was pre-processed using Cristopher Pott's [project](https://github.com/cgpotts/swda/) related to it; the pre-processed data is included in [`data/swda-acttags-and-text.csv`](data/swda-acttags-and-text.csv) for repeateability.

In [ ]:
#hide
from nbdev.showdoc import *

We first import the data from file

In [ ]:
# export
import pandas as pd

data_filepath = 'data/swda-acttags-and-text.csv'
data = pd.read_csv(data_filepath)

In [ ]:
# See the data
data.describe

<bound method NDFrame.describe of        DamslActTag                                               Text
0               ad               So, uh, describe your family budget.
1               sd  Well, I've, uh, for a lot of years I, I've pre...
2               sd  and, uh, just recently, uh, we, we set up a bu...
3               sd              and, and we're trying to stick to it.
4               sd                        We just bought a new house.
...            ...                                                ...
221611           %                                           so, yeah
221612          sd                                               I --
221613           %                                         I don't, -
221614           +  -- pretty much plan on, I mean, so far I'm pla...
221615          ba                                   That's good, uh,

[221616 rows x 2 columns]>

Divide data in test and train sets

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

data = data.dropna(axis=0)  # Drop rows with NA values
y = data.DamslActTag
X = data.Text
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

See the data division

In [ ]:
print(X_train.head(),"\n", y_train.head())

70078     really they could move right into the spare ro...
117454                                                so, -
143873           but, uh, actually not selling (( it )) , -
3186                                               Huh-uh #
180838    and I can't wait until stuff like that starts ...
Name: Text, dtype: object 
 70078     sd
117454     %
143873    sd
3186      aa
180838    sd
Name: DamslActTag, dtype: object


Convert the data labels to one-hot vectors

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

## Let's build SGNN the pipeline for processing the data

In [ ]:
#export

# CountVectorizer Parameters
char_ngram_range = (1, 4)

char_term_frequency_params = {
    'char_term_frequency__analyzer': 'char',
    'char_term_frequency__lowercase': True,
    'char_term_frequency__ngram_range': char_ngram_range,
    'char_term_frequency__strip_accents': None,
    'char_term_frequency__min_df': 2,
    'char_term_frequency__max_df': 0.99,
    'char_term_frequency__max_features': int(1e7),
}

In [ ]:
#export 
import scipy.sparse as sp
import random as rand

T = 80
d = 14
# T=80 projections for each of dimension d=14: 80 * 14 = 1120-dimensionnal word projections
rand_seeds = [rand.randint(0,T*100) for i in range(T)] # Need a different seed for each hasher

hashing_feature_union_params = {
    **{'union__sparse_random_projection_hasher_{}__n_components'.format(t): d
       for t in range(T)
    },
    **{'union__sparse_random_projection_hasher_{}__random_state'.format(t): rand_seeds[t]
       for t in range(T)
    },
    **{'union__sparse_random_projection_hasher_{}__dense_output'.format(t): False  # only AFTER hashing.
       for t in range(T)
    }
}

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.random_projection import SparseRandomProjection

params = dict()
params.update(char_term_frequency_params)
params.update(hashing_feature_union_params)

preprocessor = Pipeline([
    ("char_term_frequency", CountVectorizer()),
    ('union', FeatureUnion([
        ('sparse_random_projection_hasher_{}'.format(t), SparseRandomProjection())
        for t in range(T)
    ])),
])
preprocessor.set_params(**params)

Pipeline(steps=[('char_term_frequency',
                 CountVectorizer(analyzer='char', max_df=0.99,
                                 max_features=10000000, min_df=2,
                                 ngram_range=(1, 4))),
                ('union',
                 FeatureUnion(transformer_list=[('sparse_random_projection_hasher_0',
                                                 SparseRandomProjection(n_components=14,
                                                                        random_state=3814)),
                                                ('sparse_random_projection_hasher_1',
                                                 SparseRandomProjection(n_components=14,
                                                                        rand...
                                                 SparseRandomProjection(n_components=14,
                                                                        random_state=2806)),
                                             

In [ ]:
import tensorflow as tf
# Convert categories to one-hot-encodings, as apparently needed by keras
train_labels = tf.keras.utils.to_categorical(y_train)
test_labels = tf.keras.utils.to_categorical(y_test)

train_features = preprocessor.fit_transform(X_train)

In [ ]:
train_labels.shape

(177288, 43)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation=tf.nn.sigmoid, input_shape=(1120,)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(train_labels.shape[1], activation=tf.nn.softmax))

# Cosine annealing decay
lr_schedule = tf.keras.experimental.CosineDecay(0.025, decay_steps=1000)
# SGD optimizer with Nesterov momentum
opt = tf.keras.optimizers.SGD(nesterov=True, learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               286976    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 43)                11051     
Total params: 363,819
Trainable params: 363,819
Non-trainable params: 0
_________________________________________________________________


In [ ]:
EPOCHS=500
BATCH_SIZE=100

model.fit(train_features, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/500
177288/177288 [==============================] - 8s 43us/sample - loss: 1.8284 - acc: 0.5097
Epoch 2/500
 73900/177288 [===========>..................] - ETA: 4s - loss: 1.5874 - acc: 0.5503

KeyboardInterrupt: 

In [ ]:
test_features = preprocessor.transform(X_test)
model.evaluate(test_features, test_labels)